# Train model using tensorflow embed data 

In [1]:
# import lib
import numpy as np
import json
import pandas as pd
import os
from sklearn import svm

In [ ]:
# df = pd.read_csv("DCForTrain.csv")
# df.head()

In [2]:
path = 'cleanedDCinboxResource/'
dest_path = 'encodedDCinboxResource/'
dirs = os.listdir(path)
dirs
topics = map(lambda x: x.replace('.json.csv',''), dirs) 
topics = list(topics)
topics_df = pd.DataFrame(topics)
topics_index = list(topics_df.index)
topics_index

topic_dict = dict(zip(topics, topics_index))
topic_dict

{'healthcare': 0,
 'pet': 1,
 'business': 2,
 'fashion beauty': 3,
 'fitness sports': 4,
 'food beverage': 5,
 'media enterainment': 6,
 'lifecycle home': 7,
 'consumer': 8,
 'green cleantech': 9,
 'children education': 10,
 'software tech': 11}

In [3]:
# Set up training data
df_train = pd.DataFrame()

for i in dirs:
    temp = pd.read_csv("encodedDCinboxResource/" + i)
    temp['topic'] = i.replace('.json.csv','')
    df_train = df_train.append(temp.iloc[:temp.index.size*3//4,:])

In [4]:
# Set up for X_train
X_train = df_train[['embed']]
X_train['embed'] = X_train['embed'].apply(json.loads)
# X_train['embed']
X_train = pd.DataFrame(X_train['embed'].tolist(), columns=range(0, 512))


from sklearn.decomposition import TruncatedSVD

X_reduced = TruncatedSVD(n_components = 50, random_state = 0).fit_transform(X_train)
X_train = pd.DataFrame(X_reduced, columns=range(0,50))
# X_train = X_train.sample(frac=1,random_state = 5)


# X_train = df.copy(deep=True) # copy the dataframe 
# X_train = X_train[['embed']]
# X_train = X_train.iloc[:22981,:]
# X_train['embed'] = X_train['embed'].apply(json.loads)
# # X_train['embed']
# X_train = pd.DataFrame(X_train['embed'].tolist(), columns=range(0, 512))
# X_train.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
# Set up for y_train
y_train = df_train.copy(deep=True)
y_train = y_train[['topic']]
y_train = y_train['topic'].map(topic_dict)
y_train = y_train.reset_index(drop = True)
# y_train = y_train.sample(frac=1,random_state = 5)
y_train


0         0
1         0
2         0
3         0
4         0
         ..
34260    11
34261    11
34262    11
34263    11
34264    11
Name: topic, Length: 34265, dtype: int64

In [ ]:
# settinf up model

clf = svm.SVC()
clf.fit(X = X_train, y = y_train)
# dump the model for further usage
from joblib import dump, load
dump(clf, 'tweetModel_1_3.joblib') 


In [6]:
df_test = pd.DataFrame()

for i in dirs:
    temp = pd.read_csv("encodedDCinboxResource/" + i)
    temp['topic'] = i.replace('.json.csv','')
    df_test = df_test.append(temp.iloc[temp.index.size*3//4:,:])


# Set up for X_test
X_test = df_test[['embed']]
X_test['embed'] = X_test['embed'].apply(json.loads)
# X_train['embed']
X_test= pd.DataFrame(X_test['embed'].tolist(), columns=range(0, 512))
X_test_reduced = TruncatedSVD(n_components = 50, random_state = 0).fit_transform(X_test)
X_test = pd.DataFrame(X_test_reduced, columns=range(0,50))
X_test

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.312319,-0.242930,-0.138635,-0.051143,-0.158923,-0.147862,0.021554,0.187114,0.161063,0.071224,...,-0.074826,0.019497,-0.071345,0.043674,0.115854,-0.087908,-0.064737,0.037786,-0.056715,0.033752
1,0.373006,-0.213416,0.041770,-0.201469,-0.248171,-0.040466,0.272152,0.131102,-0.090796,-0.099057,...,-0.101098,0.009624,-0.028805,-0.085506,0.015452,-0.014385,-0.040982,-0.018764,0.084054,-0.095079
2,0.510487,-0.094853,-0.151335,0.182002,0.176556,0.009415,-0.019920,-0.024444,-0.152495,0.075457,...,-0.047901,-0.117872,-0.000486,0.009635,-0.080039,0.017950,-0.162351,0.068067,0.087630,-0.070686
3,0.464822,0.240827,-0.112117,-0.033175,0.014521,0.072294,-0.064020,-0.045689,0.116479,-0.162816,...,-0.111516,0.041188,0.069516,0.042894,-0.010494,-0.043056,-0.096400,0.063559,-0.001824,0.085464
4,0.250258,-0.105335,0.026475,-0.222507,-0.079556,0.189245,0.058676,-0.199685,-0.092474,-0.068576,...,0.092951,-0.006286,-0.023469,-0.049630,0.038451,-0.021893,0.003136,0.035309,-0.030876,-0.087558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11423,-0.038946,-0.050237,0.094512,0.134264,-0.053934,0.099420,0.040809,0.131239,-0.079947,0.040869,...,0.009343,0.025320,-0.110847,0.118098,0.020082,0.038965,0.055533,-0.029695,0.031053,0.186908
11424,0.099724,-0.036741,0.144328,0.208957,-0.001588,0.216907,0.068851,-0.126295,0.188293,0.039415,...,0.035786,-0.173692,-0.081097,-0.010720,0.025898,0.146764,-0.008567,0.011790,0.009556,0.177730
11425,0.211760,-0.084833,0.027821,0.289588,-0.167985,0.134838,-0.127141,0.057615,0.062529,-0.002108,...,0.032765,-0.039315,-0.082253,0.103295,0.025612,0.030724,-0.021126,0.005841,0.112184,0.110885
11426,0.283893,-0.131344,0.170896,-0.021222,-0.295514,0.159617,-0.060163,0.292396,0.004025,-0.076218,...,-0.081295,-0.133919,-0.123354,0.121682,-0.103700,0.107254,-0.048804,0.058093,0.086302,0.110550


In [7]:
y_test = df_test.copy(deep=True)
y_test = y_test[['topic']]
y_test = y_test['topic'].map(topic_dict)
y_test = y_test.reset_index(drop = True)
y_test

0         0
1         0
2         0
3         0
4         0
         ..
11423    11
11424    11
11425    11
11426    11
11427    11
Name: topic, Length: 11428, dtype: int64

In [13]:

# from joblib import dump, load
# clf = load('tweetModel_1_3.joblib') 

Y_predict = clf.predict(X_train.sort_index())

output=pd.DataFrame(data={"y_train": y_train.sort_index(), "Y_predict":Y_predict}) 

output.to_csv(path_or_buf="resultssvm.csv",index=False,quoting=3,sep=',')

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline

# pipeline and fit to training set
pipeline = make_pipeline(
    StandardScaler(),
    svm.SVC()
)
pipeline.fit(X=X_train, y=y_train)
    
scores = cross_val_score(pipeline, 
                         X=X_train.sort_index(),
                         y=y_train.sort_index(),
                         cv=5)  # cv refers to number of cross-validation cuts
scores

array([0.07952721, 0.07091785, 0.10331242, 0.1154239 , 0.10024807])

In [17]:
Y_predict = pipeline.predict(X=X_train.sort_index())


output=pd.DataFrame(data={"y_test": y_train.sort_index(), "Y_predict":Y_predict}) 
output.to_csv(path_or_buf="results.csv",index=False,quoting=3,sep=',')


In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train.sort_index(),Y_predict)

0.3418940609951846

In [16]:
dump(pipeline, 'tweetModelCVPipeline_1_3.joblib') 

['tweetModelCVPipeline_1_3.joblib']

In [19]:
Y_test_perdict = pipeline.predict(X=X_test)

output=pd.DataFrame(data={"y_test": y_test, "Y_predict":Y_test_perdict}) 
output.to_csv(path_or_buf="results_test.csv",index=False,quoting=3,sep=',')

In [20]:
accuracy_score(y_test,Y_test_perdict)

0.0936296814840742